Since the test data is split from training data, by finding the missing data in the training set, we can have a rough estimation of where the test data was taken.

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
training_files = list(Path('tbmData/data').glob('*.txt'))

## Diff by date

In [ ]:
from datetime import datetime
dates = sorted(datetime.strptime(str(o.stem)[-8:], '%Y%m%d') for o in training_files)
dates = pd.DataFrame({'date': dates})
diff_f = pd.concat([dates, dates.diff()], axis=1)
diff_f.columns = ['date', 'diff']
missings_f = diff.loc[diff['diff'] > pd.Timedelta('1d')]

diff_b = pd.concat([dates, dates.diff(-1)], axis=1)
diff_b.columns = ['date', 'diff']
missings_b = diff_b.loc[diff_b['diff'] < pd.Timedelta('-1d')]

missings_f

## Diff by 桩号

In [ ]:
from time import strftime

In [ ]:
fns_1 = [''.join(['CREC188_', t.strftime('%Y%m%d'), '.txt']) for t in missings_f.iloc[:,0]]
fns_2 = [''.join(['CREC188_', t.strftime('%Y%m%d'), '.txt']) for t in missings_b.iloc[:,0]]

In [ ]:
skips = []
for fn1, fn2 in zip(fns_1, fns_2):
    dfs1 = pd.read_csv('tbmData/data/'+fn1, delimiter='\t', usecols=[2])
    dfs2 = pd.read_csv('tbmData/data/'+fn2, delimiter='\t', usecols=[2])
    skips.append((dfs2.iloc[-1] - dfs1.iloc[0]).values[0])

In [ ]:
hints = pd.concat([missings_f.reset_index(drop=True),pd.Series(skips, name='skips')], axis=1)
hints